Alumno: Paez Sheridan, Pablo Santiago
Mail: pablopaezsheridan@outlook.com.ar
Ingeniero en Sistemas de Informacion

<h1>Movies</h1> 
Paez Sheridan, Pablo Santiago
Fecha: 15/07/1997
<h2>Introducción</h2>

 En este proyecto se desarrolla en Python un análisis básico de datos sobre películas de cine
 de IMDB. El set de datos que vamos a usar inicialmente se encuentra en la siguiente
 página:
 https://www.kaggle.com/datasets/carolzhangdc/imdb-5000-movie-dataset

 En ella puede encontrarse información más detallada, así como una descripción precisa de
 cada columna.

Todo: Intro

 Se plantean los apartados iniciales para su resolución sin usar la libreria pandas. Esto se
 propone así para forzar a practicar con las técnicas, herramientas y conceptos básicos de
 Python. Más adelante, hay otros apartados propuestos para su resolución con esta librería y
 otras.

Todo: Autoevaluacion


In [13]:
# Constantes globales:

MOVIES_DATA = './movie_metadata.csv'

In [7]:
# Librerías:

import csv

<h2>Parte A. Ejercicios básicos sin usar pandas </h2>

<h3> A.1. Exploración inicial básica del archivo de datos</h3>

In [16]:
def load_full_data(data_file_path):
    with open('movie_metadata.csv', mode='r') as file:
        reader = csv.reader(file)
        headers = next(reader)
        data = [row for row in reader]
    return headers, data

In [25]:
# Test de funcionamiento
full_header, full_list_data = load_full_data(MOVIES_DATA)
print(full_header)
print()
print(full_list_data[0:5])

['color', 'director_name', 'num_critic_for_reviews', 'duration', 'director_facebook_likes', 'actor_3_facebook_likes', 'actor_2_name', 'actor_1_facebook_likes', 'gross', 'genres', 'actor_1_name', 'movie_title', 'num_voted_users', 'cast_total_facebook_likes', 'actor_3_name', 'facenumber_in_poster', 'plot_keywords', 'movie_imdb_link', 'num_user_for_reviews', 'language', 'country', 'content_rating', 'budget', 'title_year', 'actor_2_facebook_likes', 'imdb_score', 'aspect_ratio', 'movie_facebook_likes']

[['Color', 'James Cameron', '723', '178', '0', '855', 'Joel David Moore', '1000', '760505847', 'Action|Adventure|Fantasy|Sci-Fi', 'CCH Pounder', 'Avatar\xa0', '886204', '4834', 'Wes Studi', '0', 'avatar|future|marine|native|paraplegic', 'http://www.imdb.com/title/tt0499549/?ref_=fn_tt_tt_1', '3054', 'English', 'USA', 'PG-13', '237000000', '2009', '936', '7.9', '1.78', '33000'], ['Color', 'Gore Verbinski', '302', '169', '563', '1000', 'Orlando Bloom', '40000', '309404152', 'Action|Adventure|F

Mostrar las posiciones de los identificadores de los campos, dados en la línea cabecera

In [18]:
list_of_enumerated_headers = [(i,header)for i,header in enumerate(full_header)]

In [19]:
# Test de funcionamiento
print(list_of_enumerated_headers)

[(0, 'color'), (1, 'director_name'), (2, 'num_critic_for_reviews'), (3, 'duration'), (4, 'director_facebook_likes'), (5, 'actor_3_facebook_likes'), (6, 'actor_2_name'), (7, 'actor_1_facebook_likes'), (8, 'gross'), (9, 'genres'), (10, 'actor_1_name'), (11, 'movie_title'), (12, 'num_voted_users'), (13, 'cast_total_facebook_likes'), (14, 'actor_3_name'), (15, 'facenumber_in_poster'), (16, 'plot_keywords'), (17, 'movie_imdb_link'), (18, 'num_user_for_reviews'), (19, 'language'), (20, 'country'), (21, 'content_rating'), (22, 'budget'), (23, 'title_year'), (24, 'actor_2_facebook_likes'), (25, 'imdb_score'), (26, 'aspect_ratio'), (27, 'movie_facebook_likes')]


<h3>A.2. Campos principales de una película</h3>

Los campos (columnas) del archivo son demasiados. No nos interesan todos ellos. Dada
una lista con todos los campos, se pide extraer otra lista sólo con los campos con los que
vamos a trabajar en los siguientes apartados: movie_title , title_year ,
director_name , actor_1_name , language , country , color , budget ,
imdb_score y movie_imdb_link 

In [82]:
def main_data_from_item(data:list[str]) -> list[str]:
    selected_columns = ["movie_title", "title_year", "director_name", "actor_1_name", "language", "country", "budget", "imdb_score", "movie_imdb_link"]
    selected_columns_indexes = [full_header.index(d) for c in selected_columns for d in full_header if c in d]
    return [data[si] for si in selected_columns_indexes]    

In [83]:
# Test de funcionamiento
print(main_data_from_item(full_header))
print()
datos_avatar_2009 = main_data_from_item(full_list_data[0])
print(datos_avatar_2009)
print()
datos_star_wars_7 = main_data_from_item(full_list_data[4])
print(datos_star_wars_7)

['movie_title', 'title_year', 'director_name', 'actor_1_name', 'language', 'country', 'budget', 'imdb_score', 'movie_imdb_link']

['Avatar\xa0', '2009', 'James Cameron', 'CCH Pounder', 'English', 'USA', '237000000', '7.9', 'http://www.imdb.com/title/tt0499549/?ref_=fn_tt_tt_1']

['Star Wars: Episode VII - The Force Awakens\xa0            ', '', 'Doug Walker', 'Doug Walker', '', '', '', '7.1', 'http://www.imdb.com/title/tt5289954/?ref_=fn_tt_tt_1']


<h3>A.3. Algunos ajustes en los campos</h3>

Observa la anomalía en el string del título de la película. También, queremos tratar algunos
campos como numéricos, ya sea enteros (el año y el presupuesto) o reales (la valoración).
(Algunos campos numéricos enteros están vacíos en el archivo de datos; para ellos, el valor
imputado será -1 .) Además, en las urls de las películas no necesitaremos el fragmento
final, iniciado con ?ref_ 

In [88]:
def datatypes_arranged(data : list[str])->list[str]:
    data_treated = []
    numeric_indexes = [1,6,7]
    for index,d in enumerate(data):
        try:
            if d == "" and index in numeric_indexes:
                data_treated.append(-1)
            elif (index in numeric_indexes):
                data_treated.append(float(d))
            elif (index == 8):
                data_treated.append(d.split('?ref_')[0])
            else:
                data_treated.append(d)
        except:
            data_treated.append(d)
    return data_treated

In [89]:
# Test de funcionamiento
print(datatypes_arranged(datos_avatar_2009))
print(datatypes_arranged(datos_star_wars_7))

['Avatar\xa0', 2009.0, 'James Cameron', 'CCH Pounder', 'English', 'USA', 237000000.0, 7.9, 'http://www.imdb.com/title/tt0499549/']
['Star Wars: Episode VII - The Force Awakens\xa0            ', -1, 'Doug Walker', 'Doug Walker', '', '', -1, 7.1, 'http://www.imdb.com/title/tt5289954/']


<h3> A.4. Recuperación de alguna información básica</h3>

<p> Diseña funciones para recuperar la siguiente información: </p>
<ul>
    <li> El conjunto de valores posibles del campo `Color`.</li>
    <li> Los títulos de película de nuestro archivo (limpios de caracteres extraños), junto con la cantidad de calificadores (un entero), que contengan la subcadena "Victor".</li>        
<ul>

In [102]:
def column_values(column_index: int)-> set:
    values = set()
    column_data = [row[column_index] for row in full_list_data[1:]] 
    for d in column_data:
        values.add(d)
    return values


colores = column_values(0)

In [103]:
# Test de funcionamiento
print(colores)
# print(pelis_victor)

{'', 'Color', ' Black and White'}


<h2> B Datos en un diccionario [2 puntos]</h2>
<h3>B.1. Carga únicamente de datos que vamos a usar</h3>
<p> En lugar de cargar todos los datos del archivo, luego generar otro archivo con los datos que
 nos interesan y luego ajustarlos, es mejor diseñar una función que recorre el archivo de
 datos y, de cada línea del archivo, únicamente retiene la parte que nos interesa, cargando la
 cabecera por un lado y, por otro, la lista de datos de interés, ajustados como ya hemos
 dicho.
 Cargaremos esta información en un diccionario, cuyas claves serán pares (título, año)
 y cuyos valores se recogen en una lista con el resto de los campos.
 Además, los datos de cada película tras la clave (título, año) están repetidas en el
 archivo de datos con todos los datos idénticos; y lo que es peor: alguna que está repetida</p>

In [ ]:
# Test de funcionamiento
main_header, main_dict_data = load_main_data(MOVIES_DATA)
print(main_header)
print()
for title_year, pieces in list(main_dict_data.items())[:5]:
    print(title_year, " -> ", pieces)